# 1. はじめに
midiファイルを元に、音の高さの動きを把握する。
word2vecの要領で、どの音とどの音が近いか、みたいなことをやって見る。

# 2. データセットについて
- http://colinraffel.com/projects/lmd/#get

# 3. コード

下記を参考
- https://www.tensorflow.org/tutorials/word2vec
-  https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/examples/tutorials/word2vec/word2vec_basic.py

In [1]:
import collections
import math
import os
import random
import zipfile
import glob
import datetime
from itertools import chain

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [2]:
files_name = glob.glob("./input/lmd_matched/*/*/*/*/*.mid")

In [3]:
import pretty_midi

# 10000曲のみ使用
files_name = files_name[:10000]

notes_list = []

# 楽器の絞り込み
instrument_program_list = [1,2,3,4,5,6]

# 
success = 0
failure = 0
for i,file_name in enumerate(files_name):
    try:
        midi_data = pretty_midi.PrettyMIDI(file_name)
    except:
        failure += 1
        continue
    for instrument in midi_data.instruments:
        # 特定の楽器のみ、音を取得する
        if instrument.program in instrument_program_list:
            notes = []
            for note in instrument.notes:
                notes.append(note.pitch)
            notes_list.append(notes)
            success+=1
    if i % 10 == 0:
        print("成功instrument数:",success,"失敗midi数：",failure)

成功instrument数: 3 失敗midi数： 0


/Users/takizawa/dev/python/music/lib/python3.5/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


成功instrument数: 9 失敗midi数： 0
成功instrument数: 11 失敗midi数： 0
成功instrument数: 15 失敗midi数： 0
成功instrument数: 19 失敗midi数： 0
成功instrument数: 21 失敗midi数： 0
成功instrument数: 26 失敗midi数： 0
成功instrument数: 31 失敗midi数： 1
成功instrument数: 33 失敗midi数： 1
成功instrument数: 42 失敗midi数： 1
成功instrument数: 50 失敗midi数： 2
成功instrument数: 56 失敗midi数： 2
成功instrument数: 68 失敗midi数： 2
成功instrument数: 71 失敗midi数： 2
成功instrument数: 79 失敗midi数： 2
成功instrument数: 82 失敗midi数： 3
成功instrument数: 83 失敗midi数： 3
成功instrument数: 94 失敗midi数： 3
成功instrument数: 103 失敗midi数： 3
成功instrument数: 110 失敗midi数： 3
成功instrument数: 122 失敗midi数： 6
成功instrument数: 174 失敗midi数： 6
成功instrument数: 185 失敗midi数： 6
成功instrument数: 190 失敗midi数： 6
成功instrument数: 195 失敗midi数： 6
成功instrument数: 200 失敗midi数： 6
成功instrument数: 211 失敗midi数： 6
成功instrument数: 212 失敗midi数： 6
成功instrument数: 214 失敗midi数： 6
成功instrument数: 218 失敗midi数： 6
成功instrument数: 221 失敗midi数： 7
成功instrument数: 232 失敗midi数： 7
成功instrument数: 246 失敗midi数： 7
成功instrument数: 254 失敗midi数： 7
成功instrument数: 257 失敗midi数： 

In [4]:
from itertools import chain
note_list_flatten = list(chain.from_iterable(notes_list))
# note_list_flatten = [str(x) for x in note_list_flatten]

In [5]:
def build_dataset(words, n_words):
# def build_dataset(words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
#     count = []
    count.extend(collections.Counter(words).most_common(n_words))
#     count.extend(collections.Counter(words))
#     print(count)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(note_list_flatten,80)

In [6]:
count

[['UNK', 1184],
 (62, 223990),
 (64, 213430),
 (60, 191270),
 (69, 186337),
 (67, 181619),
 (57, 170980),
 (65, 146374),
 (59, 139893),
 (66, 124760),
 (55, 123116),
 (61, 119210),
 (63, 117689),
 (72, 102116),
 (58, 95254),
 (71, 95126),
 (74, 93227),
 (70, 82897),
 (54, 79021),
 (56, 78583),
 (42, 75769),
 (68, 74002),
 (76, 71630),
 (52, 64251),
 (53, 59179),
 (50, 50926),
 (79, 46619),
 (51, 45715),
 (73, 41920),
 (77, 40036),
 (43, 39761),
 (81, 39564),
 (36, 39084),
 (45, 38735),
 (48, 37481),
 (44, 36187),
 (75, 33823),
 (78, 31605),
 (40, 30978),
 (38, 30644),
 (49, 28220),
 (46, 25927),
 (41, 23795),
 (83, 21875),
 (47, 19668),
 (35, 18376),
 (82, 18135),
 (86, 16582),
 (80, 16163),
 (84, 15735),
 (37, 15392),
 (39, 13585),
 (88, 12438),
 (85, 9869),
 (31, 6884),
 (33, 6368),
 (93, 5540),
 (89, 5467),
 (87, 4707),
 (90, 4421),
 (34, 4397),
 (32, 4058),
 (29, 3801),
 (92, 3739),
 (91, 3235),
 (95, 2664),
 (28, 2067),
 (30, 1962),
 (94, 1201),
 (27, 1199),
 (25, 953),
 (26, 848)

In [7]:
note_to_str_dictionary = dict()
note_to_str_dictionary['UNK'] = 'UNK'
for i in range(int(128/12)+1):
    note_to_str_dictionary[i*12+0] = "C"+str(i-1)
    note_to_str_dictionary[i*12+1] = "C#"+str(i-1)
    note_to_str_dictionary[i*12+2] = "D"+str(i-1)
    note_to_str_dictionary[i*12+3] = "D#"+str(i-1)
    note_to_str_dictionary[i*12+4] = "E"+str(i-1)
    note_to_str_dictionary[i*12+5] = "F"+str(i-1)
    note_to_str_dictionary[i*12+6] = "F#"+str(i-1)
    note_to_str_dictionary[i*12+7] = "G"+str(i-1)
    if i < 10:
        note_to_str_dictionary[i*12+8] = "G#"+str(i-1)
        note_to_str_dictionary[i*12+9] = "A"+str(i-1)
        note_to_str_dictionary[i*12+10] = "A#"+str(i-1)
        note_to_str_dictionary[i*12+11] = "B"+str(i-1)

In [8]:
note_to_str_dictionary

{0: 'C-1',
 1: 'C#-1',
 2: 'D-1',
 3: 'D#-1',
 4: 'E-1',
 5: 'F-1',
 6: 'F#-1',
 7: 'G-1',
 8: 'G#-1',
 9: 'A-1',
 10: 'A#-1',
 11: 'B-1',
 12: 'C0',
 13: 'C#0',
 14: 'D0',
 15: 'D#0',
 16: 'E0',
 17: 'F0',
 18: 'F#0',
 19: 'G0',
 20: 'G#0',
 21: 'A0',
 22: 'A#0',
 23: 'B0',
 24: 'C1',
 25: 'C#1',
 26: 'D1',
 27: 'D#1',
 28: 'E1',
 29: 'F1',
 30: 'F#1',
 31: 'G1',
 32: 'G#1',
 33: 'A1',
 34: 'A#1',
 35: 'B1',
 36: 'C2',
 37: 'C#2',
 38: 'D2',
 39: 'D#2',
 40: 'E2',
 41: 'F2',
 'UNK': 'UNK',
 43: 'G2',
 44: 'G#2',
 45: 'A2',
 46: 'A#2',
 47: 'B2',
 48: 'C3',
 49: 'C#3',
 50: 'D3',
 51: 'D#3',
 52: 'E3',
 53: 'F3',
 54: 'F#3',
 55: 'G3',
 56: 'G#3',
 57: 'A3',
 58: 'A#3',
 59: 'B3',
 60: 'C4',
 61: 'C#4',
 62: 'D4',
 63: 'D#4',
 64: 'E4',
 65: 'F4',
 66: 'F#4',
 67: 'G4',
 68: 'G#4',
 69: 'A4',
 70: 'A#4',
 71: 'B4',
 72: 'C5',
 73: 'C#5',
 74: 'D5',
 75: 'D#5',
 76: 'E5',
 77: 'F5',
 78: 'F#5',
 79: 'G5',
 80: 'G#5',
 81: 'A5',
 82: 'A#5',
 83: 'B5',
 84: 'C6',
 85: 'C#6',
 86: 'D6',
 8

In [9]:
reverse_dictionary

{0: 'UNK',
 1: 62,
 2: 64,
 3: 60,
 4: 69,
 5: 67,
 6: 57,
 7: 65,
 8: 59,
 9: 66,
 10: 55,
 11: 61,
 12: 63,
 13: 72,
 14: 58,
 15: 71,
 16: 74,
 17: 70,
 18: 54,
 19: 56,
 20: 42,
 21: 68,
 22: 76,
 23: 52,
 24: 53,
 25: 50,
 26: 79,
 27: 51,
 28: 73,
 29: 77,
 30: 43,
 31: 81,
 32: 36,
 33: 45,
 34: 48,
 35: 44,
 36: 75,
 37: 78,
 38: 40,
 39: 38,
 40: 49,
 41: 46,
 42: 41,
 43: 83,
 44: 47,
 45: 35,
 46: 82,
 47: 86,
 48: 80,
 49: 84,
 50: 37,
 51: 39,
 52: 88,
 53: 85,
 54: 31,
 55: 33,
 56: 93,
 57: 89,
 58: 87,
 59: 90,
 60: 34,
 61: 32,
 62: 29,
 63: 92,
 64: 91,
 65: 95,
 66: 28,
 67: 30,
 68: 94,
 69: 27,
 70: 25,
 71: 26,
 72: 96,
 73: 127,
 74: 98,
 75: 97,
 76: 24,
 77: 13,
 78: 23,
 79: 103,
 80: 106}

In [10]:
data_index = 0
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [11]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], note_to_str_dictionary[reverse_dictionary[batch[i]]],
          '->', labels[i, 0],note_to_str_dictionary[reverse_dictionary[labels[i, 0]]])

2 E4 -> 4 A4
2 E4 -> 4 A4
4 A4 -> 2 E4
4 A4 -> 2 E4
2 E4 -> 4 A4
2 E4 -> 4 A4
4 A4 -> 2 E4
4 A4 -> 2 E4


In [12]:
data_index

7

In [13]:
# Step 4: Build and train a skip-gram model.

# tensorflowチュートリアルの説明より、16~512に収めるとのこと。
batch_size = 128
# embedding_size = 128  # Dimension of the embedding vector.
# MIDIノートは0〜127。
embedding_size = 5
skip_window = 1      # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
# valid_window = 100  # Only pick dev samples in the head of the distribution.

# valid_examples = np.random.choice(valid_window, valid_size, replace=False)
s = set(data)
valid_examples = random.sample(s,valid_size)
vocabulary_size = len(s)

# num_sampled = 64    # Number of negative examples to sample.
num_sampled = 10

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                       num_sampled, vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    ## 学習率は変更した
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    # コサイン類似度で、ミニバッチのデータとembedding
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    
    # valid_datasetはランダムに選択されたvalid_examples
    valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [16]:
# Step 5: Begin training.
num_steps = 100000

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        # バッチデータを作成
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        # feed_dictに辞書型で入れる
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        
        # 2000回ごとにlossの平均を表示
        loss_interval = 2000
        if step % loss_interval == 0:
            if step > 0:
                average_loss /= loss_interval
            # The average los s is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            # 表示したら0にする。
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = note_to_str_dictionary[reverse_dictionary[valid_examples[i]]]
                top_k = 8 # number of nearest neighbors
                
                # simはsimilarity
                # similarityはvalid_embeddingとnormalized_embeddingの掛け算
                # 内積を取っているようだ。
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = note_to_str_dictionary[reverse_dictionary[nearest[k]]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

# Step 6: Visualize the embeddings.

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')

    plt.savefig(filename)

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = 128
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
#     labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    labels = [note_to_str_dictionary[reverse_dictionary[i]] for i in set(data)]
    
    # 保存する画像のファイル名には時間を記載する
    now = datetime.datetime.now()
    file_time = now.strftime("%Y%m%d%H%M%S")
    plot_with_labels(low_dim_embs, labels,filename="tsne_"+file_time+".png")

except ImportError:
    print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

Initialized
Average loss at step  0 :  19.3876132965
Nearest to A#6: B6, G5, D6, D7, G#1, D#2, G#2, A2,
Nearest to F2: D#3, E2, F4, D2, F#3, A#4, C#5, D#5,
Nearest to G9: G#5, A#4, D#3, D#5, A2, C2, C6, D2,
Nearest to D3: A1, B4, A#2, D7, A4, G#2, B3, F#4,
Nearest to E1: C#0, F#1, A#1, B4, C#1, D#4, A3, C6,
Nearest to A5: B0, E6, D#2, F#3, G#6, B2, A#7, G6,
Nearest to C#0: C6, E1, D#4, F6, F#1, C2, A3, D1,
Nearest to A3: C#7, C#4, C#1, C#0, C#3, G#2, D3, E1,
Nearest to C#4: B0, A#3, A3, D#1, C#3, C#1, G#3, C#7,
Nearest to F1: F4, C1, F#1, A#1, C7, G6, G1, C#5,
Nearest to G#1: G5, D#2, B6, B2, C#6, G#6, E2, G6,
Nearest to C4: A#3, C#5, D#1, A#4, C3, UNK, A3, A1,
Nearest to C#7: A3, G#2, D7, C#3, G4, C#1, C#0, C#5,
Nearest to G2: F#4, D4, D#2, B2, A2, C5, E6, A#6,
Nearest to UNK: A#4, C#1, A#5, F#6, D2, F#3, D#4, C6,
Nearest to G5: B2, G#1, D#2, A#6, C5, G#6, B6, A#2,
Average loss at step  2000 :  3.67314747971
Average loss at step  4000 :  2.71543237671
Average loss at step  6000 :  2.7